1. Preprocessing

In [11]:
import pandas as pd
import numpy as np
import re
import string
import nltk

In [12]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
df = pd.read_csv('/content/tweets.csv')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [14]:
df['clean_text'] = df['text'].apply(clean_text)
display(df.head())

,id,keyword,location,text,target,clean_text
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Ston...",1,communal violence bhainsa telangana stone pelt...
1,1,ablaze,NaN,Telangana: Section 144 has been imposed in Bha...,1,telangana section imposed bhainsa january clas...
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https:...,1,arsonist set car ablaze dealership
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pard...",0,lord jesus love brings freedom pardon fill hol...


In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'\d+', '', text)             # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    tokens = nltk.word_tokenize(text)           # tokenize
    tokens = [word for word in tokens if word not in stop_words]  # remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]      # lemmatization
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(clean_text)

2. Feature Extraction

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X = tfidf.fit_transform(df['clean_text']).toarray()
y = df['target'].values


 Deep Learning Models

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

mlp = Sequential()
mlp.add(Dense(128, input_dim=X.shape[1], activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(64, activation='relu'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1, activation='sigmoid'))

mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

mlp.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8021 - loss: 0.5048 - val_accuracy: 0.8953 - val_loss: 0.3046
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.8995 - loss: 0.2620 - val_accuracy: 0.8949 - val_loss: 0.2684
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9368 - loss: 0.1695 - val_accuracy: 0.8861 - val_loss: 0.2799
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9517 - loss: 0.1262 - val_accuracy: 0.8953 - val_loss: 0.3062
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9672 - loss: 0.0937 - val_accuracy: 0.8865 - val_loss: 0.3442
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.9815 - loss: 0.0618 - val_accuracy: 0.8865 - val_loss: 0.3838
Epoch 7/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.9863 - loss: 0.0489 - val_accuracy: 0.8861 - val_loss: 0.4266
Epoch 8/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9877 - loss: 0.0358 - val_accu

In [19]:
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val_cnn = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)


In [20]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

cnn = Sequential()
cnn.add(Conv1D(64, 3, activation='relu', input_shape=(X.shape[1], 1)))
cnn.add(GlobalMaxPooling1D())
cnn.add(Dense(64, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

cnn.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_data=(X_val_cnn, y_val))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 45s 154ms/step - accuracy: 0.8077 - loss: 0.5332 - val_accuracy: 0.8259 - val_loss: 0.4497
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 43s 151ms/step - accuracy: 0.8042 - loss: 0.4868 - val_accuracy: 0.8259 - val_loss: 0.4498
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 81s 150ms/step - accuracy: 0.8142 - loss: 0.4696 - val_accuracy: 0.8259 - val_loss: 0.4417
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 81s 147ms/step - accuracy: 0.8137 - loss: 0.4645 - val_accuracy: 0.8259 - val_loss: 0.4407
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 83s 150ms/step - accuracy: 0.8073 - loss: 0.4767 - val_accuracy: 0.8259 - val_loss: 0.4415
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 81s 148ms/step - accuracy: 0.8087 - loss: 0.4726 - val_accuracy: 0.8259 - val_loss: 0.4439
Epoch 7/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 42s 146ms/step - accuracy: 0.8130 - loss: 0.4684 - val_accuracy: 0.8259 - val_loss: 0.4468
Epoch 8/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 80s 141ms/step - accuracy: 0.8059 - loss: 0

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional

# Tokenize
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['clean_text'])
X_seq = tokenizer.texts_to_sequences(df['clean_text'])
X_pad = pad_sequences(X_seq, maxlen=100)

X_train_seq, X_val_seq, y_train_seq, y_val_seq = train_test_split(X_pad, y, test_size=0.2, random_state=42)

# Model
bi_lstm = Sequential()
bi_lstm.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
bi_lstm.add(Bidirectional(LSTM(64, return_sequences=False)))
bi_lstm.add(Dense(64, activation='relu'))
bi_lstm.add(Dropout(0.5))
bi_lstm.add(Dense(1, activation='sigmoid'))

bi_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

bi_lstm.fit(X_train_seq, y_train_seq, epochs=10, batch_size=32, validation_data=(X_val_seq, y_val_seq))


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


285/285 ━━━━━━━━━━━━━━━━━━━━ 45s 133ms/step - accuracy: 0.8198 - loss: 0.4471 - val_accuracy: 0.8993 - val_loss: 0.2623
Epoch 2/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 39s 128ms/step - accuracy: 0.9256 - loss: 0.1991 - val_accuracy: 0.8901 - val_loss: 0.2709
Epoch 3/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 43s 133ms/step - accuracy: 0.9579 - loss: 0.1298 - val_accuracy: 0.8931 - val_loss: 0.3095
Epoch 4/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 42s 136ms/step - accuracy: 0.9773 - loss: 0.0702 - val_accuracy: 0.8808 - val_loss: 0.4330
Epoch 5/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 39s 129ms/step - accuracy: 0.9871 - loss: 0.0418 - val_accuracy: 0.8813 - val_loss: 0.5643
Epoch 6/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 37s 129ms/step - accuracy: 0.9920 - loss: 0.0245 - val_accuracy: 0.8769 - val_loss: 0.6658
Epoch 7/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 40s 126ms/step - accuracy: 0.9953 - loss: 0.0156 - val_accuracy: 0.8773 - val_loss: 0.7246
Epoch 8/10
285/285 ━━━━━━━━━━━━━━━━━━━━ 37s 130ms/step - accuracy: 0.9961 - loss: 0.0130 - val

Evaluation Metrics

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, X_data, y_true):
    y_pred = (model.predict(X_data) > 0.5).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return acc, prec, rec, f1


In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_val)

# SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_val)

# Metrics for both
metrics_lr = (accuracy_score(y_val, y_pred_lr), precision_score(y_val, y_pred_lr),
              recall_score(y_val, y_pred_lr), f1_score(y_val, y_pred_lr))

metrics_svm = (accuracy_score(y_val, y_pred_svm), precision_score(y_val, y_pred_svm),
               recall_score(y_val, y_pred_svm), f1_score(y_val, y_pred_svm))


In [24]:
# For MLP
acc, prec, rec, f1 = evaluate_model(mlp, X_val, y_val)
print("MLP Results:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}\n")

# For CNN
acc, prec, rec, f1 = evaluate_model(cnn, X_val_cnn, y_val)
print("CNN Results:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}\n")

# For Bi-LSTM
acc, prec, rec, f1 = evaluate_model(bi_lstm, X_val_seq, y_val_seq)
print("Bi-LSTM Results:")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}\n")

# For Logistic Regression
print("Logistic Regression Results:")
print(f"Accuracy: {metrics_lr[0]:.4f}")
print(f"Precision: {metrics_lr[1]:.4f}")
print(f"Recall: {metrics_lr[2]:.4f}")
print(f"F1-score: {metrics_lr[3]:.4f}\n")

# For SVM
print("SVM Results:")
print(f"Accuracy: {metrics_svm[0]:.4f}")
print(f"Precision: {metrics_svm[1]:.4f}")
print(f"Recall: {metrics_svm[2]:.4f}")
print(f"F1-score: {metrics_svm[3]:.4f}\n")


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
MLP Results:
Accuracy: 0.8835
Precision: 0.6719
Recall: 0.6465
F1-score: 0.6589

72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step
CNN Results:
Accuracy: 0.8259
Precision: 0.0000
Recall: 0.0000
F1-score: 0.0000



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step
Bi-LSTM Results:
Accuracy: 0.8861
Precision: 0.6974
Recall: 0.6111
F1-score: 0.6514

Logistic Regression Results:
Accuracy: 0.8848
Precision: 0.8895
Recall: 0.3864
F1-score: 0.5387

SVM Results:
Accuracy: 0.8993
Precision: 0.8199
Recall: 0.5404
F1-score: 0.6514

